In [1]:
# Importing libraries
import pandas as pd
import pymysql
from sqlalchemy import  create_engine

In [18]:
# Load data
df = pd.read_csv('highest_grossing_films_clean.csv')
df.head()

,Position,Peak,Title,Revenue,Year
0,1,1,Avatar,"$2,923,706,026",2009
1,2,1,Avengers: Endgame,"$2,797,501,328",2019
2,3,3,Avatar: The Way of Water,"$2,320,250,281",2022
3,4,1,Titanic,"T$2,257,844,554",1997
4,5,3,Star Wars: The Force Awakens,"$2,068,223,624",2015


In [19]:
# MySQL Connection
conn = pymysql.Connect(
    host='localhost',
    port=3307,
    password='filmsuserpassword',
    user='filmsuser',
    database='filmsdb'
)

In [20]:
cursor = conn.cursor()

In [21]:
# Query
cursor.execute("SHOW DATABASES;")
cursor.fetchall()

(('filmsdb',), ('information_schema',), ('performance_schema',))

In [22]:
create_query_file = open('./sql/mysql_create_table.sql')
create_query = create_query_file.read()
create_query

'CREATE TABLE films(\n    Position INT,\n    Peak VARCHAR(255),\n    Title VARCHAR(255),\n    Revenue VARCHAR(255),\n    Year INT\n);'

In [23]:
cursor.execute(create_query)

0

In [29]:
data = list(df.itertuples(index=None, name=None))

In [26]:
engine = create_engine("mysql+pymysql://", creator=lambda: conn)

In [27]:
pd.read_sql("SELECT * FROM films;", con=conn)

C:\Users\PC\AppData\Local\Temp\ipykernel_14980\4082398294.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT * FROM films;", con=conn)


,Position,Peak,Title,Revenue,Year


### Load data into database

In [46]:
merge_query_file = open('./sql/mysql_upsert.sql')
merge_query = merge_query_file.read()
merge_query

'INSERT INTO films (Position, Peak, Title, Revenue, Year)\nVALUES (%s, %s, %s, %s, %s)\nON DUPLICATE KEY UPDATE\n    Peak = VALUES(Peak),\n    Title = VALUES(Title),\n    Revenue = VALUES(Revenue),\n    Year = VALUES(Year);\n\n'

In [47]:
cursor.executemany(merge_query, data)

50

In [48]:
conn.commit()

In [49]:
pd.read_sql("SELECT * FROM films;", con=engine)

,Position,Peak,Title,Revenue,Year
0,1,1,Avatar,"$2,923,706,026",2009
1,2,1,Avengers: Endgame,"$2,797,501,328",2019
2,3,3,Avatar: The Way of Water,"$2,320,250,281",2022
3,4,1,Titanic,"T$2,257,844,554",1997
4,5,3,Star Wars: The Force Awakens,"$2,068,223,624",2015
...,...,...,...,...,...
95,46,1,Jurassic Park,"$1,037,535,230",1993
96,47,24,Despicable Me 3,"$1,034,800,131",2017
97,48,22,Finding Dory,"$1,028,570,942",2016
98,49,5,Alice in Wonderland,"$1,025,468,216",2010
